In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters, likelihood_ratio
from tensorflow.python.framework.ops import disable_eager_execution

2022-08-19 10:40:14.773385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 10:40:14.773440: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 10000
angles = np.array([1.53, 1.67])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)


# Likelihood Ratio of Data
l_out = likelihood_ratio(x_train)

In [9]:
# model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')

# for l in model_interpolate.layers:
#     l.trainable=False
    
# loss_fn = tf.keras.losses.BinaryCrossentropy()

# # Building model_angles which is used to train (theta, phi)
# inputs_hold = tf.keras.Input(shape=(1,))
# simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
# model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# # Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
# raw_inputs = tf.keras.Input(shape=(2,))
# inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
# output = model_interpolate(inputs)

# model_paramfinder = Model(inputs = raw_inputs, outputs = output)
# model_paramfinder.compile(loss=loss_fn, optimizer='Adam')
# model_paramfinder.summary()


# Likelihood Version
# disable_eager_execution()

loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = likelihood_ratio(inputs)

model_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_paramfinder.compile(loss=loss_fn, optimizer='Adam')
model_paramfinder.summary()



epochs, iterations, batch_size = 1, 50, 500

e, b = 1, 50
x_inputs = x_train[:, 0:2]
    
for i in range(40):
    model_paramfinder.fit(x_inputs, y_train, epochs=e, batch_size=b)
    print(i,"Fitted result: ", model_paramfinder.trainable_weights[:])
#     x = model_pf(x_inputs).numpy().transpose()[0]

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like_1 (TFOpLambda)     (None, 2)            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_7 (Sli (None, 1)            0           tf.ones_like_1[0][0]             
__________________________________________________________________________________________________
model_2 (Functional)            (None, 2)            2           tf.__operators__.getitem_7[0][0] 
____________________________________________________________________________________________

320/320 [==============================] - 1s 1ms/step - loss: 0.6486
0 Fitted result:  [<tf.Variable 'dense_1/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[1.1854151, 1.1919097]], dtype=float32)>]
320/320 [==============================] - 0s 1ms/step - loss: 0.6405
1 Fitted result:  [<tf.Variable 'dense_1/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[1.2926296, 1.3339001]], dtype=float32)>]
320/320 [==============================] - 0s 2ms/step - loss: 0.6374
2 Fitted result:  [<tf.Variable 'dense_1/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[1.3603923, 1.4250245]], dtype=float32)>]
320/320 [==============================] - 0s 1ms/step - loss: 0.6361
3 Fitted result:  [<tf.Variable 'dense_1/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[1.4043701, 1.4924622]], dtype=float32)>]
320/320 [==============================] - 0s 1ms/step - loss: 0.6356
4 Fitted result:  [<tf.Variable 'dense_1/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[1.4417433, 1.5319744]

In [22]:
print(model_paramfinder(x_inputs))
print(likelihood_ratio(x_train))
print(y_train)

tf.Tensor([0.414758   0.7672632  0.61693394 ... 0.8213023  0.65844995 0.56007206], shape=(16000,), dtype=float32)
tf.Tensor([0.3639371  0.78642594 0.59208967 ... 0.85877816 0.66451228 0.5238565 ], shape=(16000,), dtype=float64)
[0. 1. 0. ... 1. 0. 1.]


In [ ]:
test_on_integers(x_train, y_train)

In [10]:
epochs, iterations, batch_size = 1, 50, 100

weights = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
angle_errors = (angles - weights) / angles
average_error = np.sum(np.abs(angle_errors)) / 2
print('Average error %:', average_error * 100)

160/160 [==============================] - 1s 2ms/step - loss: 0.6350
0 Fitted result:  tf.Tensor([1.5920904 1.6547896], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6349
1 Fitted result:  tf.Tensor([1.5936916 1.6543394], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6350
2 Fitted result:  tf.Tensor([1.5958605 1.6524491], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6350
3 Fitted result:  tf.Tensor([1.5945164 1.6541998], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6350
4 Fitted result:  tf.Tensor([1.5948465 1.6560528], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6349
5 Fitted result:  tf.Tensor([1.5921872 1.6540906], shape=(2,), dtype=float32)
160/160 [==============================] - 0s 2ms/step - loss: 0.6350
6 Fitted result:  tf.Tensor([1.5923917 1.6

In [33]:
# xr = model_interpolate(x_train).numpy().transpose()[0]
# y = model_interpolate(x_train)
# loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
# print(loss)

# x_train_mod = np.copy(x_train)
# x_train_mod[:, 2] = x_train[:, 2]
# x_train_mod[:, 3] = x_train[:, 3] - .1
# xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
# y_mod = model_interpolate(x_train_mod)
# loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
# print(loss_mod)

0.6019409317687167


In [7]:
xr = likelihood_ratio(x_train).numpy().transpose()
y = likelihood_ratio(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2] - .05
x_train_mod[:, 3] = x_train[:, 3] - .05
xr_mod = likelihood_ratio(x_train_mod).numpy().transpose()
y_mod = likelihood_ratio(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)

0.6263753486503797
0.6265406172742207


In [14]:
likelihood_ratio(x_train)[0]

<tf.Tensor: shape=(), dtype=float64, numpy=0.3996452546492857>